## Import Library

In [1]:
import gzip
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.layers import GRU, Dense, TimeDistributed, Input, \
    Bidirectional, Embedding, Dropout, MultiHeadAttention, \
        Add, LayerNormalization, BatchNormalization

from keras.models import Sequential, Model, load_model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model, model_to_dot

from tensorflow.keras.regularizers import L1, L2



## Load Data

In [2]:
def prepareData ():
    f = gzip.GzipFile('../Data/Preprocessed/CullPDB_6133_filtered_profile_MASK.npy.gz', "r")
    CullPDB_profile = np.load(f)
    f.close()

    f = gzip.GzipFile('../Data/Preprocessed/CullPDB_6133_filtered_sequence_MASK.npy.gz', "r")
    CullPDB_sequence = np.load(f)
    f.close()

    ## CullPDB_sequence_integer : transform one hot encoding into a single integer for Embedding input
    CullPDB_sequence_integer = np.zeros((CullPDB_sequence.shape[0], CullPDB_sequence.shape[1]))
    print("Shape of amino acid sequence (before transform):", CullPDB_sequence.shape)
    for i in range(0, CullPDB_sequence_integer.shape[0]):
        for j in range(0, CullPDB_sequence_integer.shape[1]):
            CullPDB_sequence_integer[i][j] = np.argmax(CullPDB_sequence[i][j])
    print("Shape of amino acid sequence (after transform):", CullPDB_sequence_integer.shape)

    f = gzip.GzipFile("../Data/Preprocessed/CullPDB_6133_filtered_traininglabel_MASK.npy.gz", "r")
    CullPDB_traininglabel = np.load(f)
    f.close()

    hmm_profile = np.load("../Data/Preprocessed/hhm_train.npy", "r")

    CullPDB_data = {"sequence": CullPDB_sequence_integer, "profile": CullPDB_profile, "label":CullPDB_traininglabel, "hmm_profile":hmm_profile}

    return CullPDB_data

In [3]:
data = prepareData()
print("Shape of PSSM profile:", data["profile"].shape)
print("Shape of HMM profile:", data["hmm_profile"].shape)
print("Shape of Label:", data["label"].shape)

Shape of amino acid sequence (before transform): (5534, 700, 22)
Shape of amino acid sequence (after transform): (5534, 700)
Shape of PSSM profile: (5534, 700, 21)
Shape of HMM profile: (5534, 700, 20)
Shape of Label: (5534, 700, 9)


## Explanatory Data Analysis

### Source: 

<a href="https://github.com/taneishi/CB513_dataset">
    <svg height="64" aria-hidden="true" viewBox="0 0 24 24" version="1.1" width="64" data-view-component="true" class="octicon octicon-mark-github">
        <path d="M12.5.75C6.146.75 1 5.896 1 12.25c0 5.089 3.292 9.387 7.863 10.91.575.101.79-.244.79-.546 0-.273-.014-1.178-.014-2.142-2.889.532-3.636-.704-3.866-1.35-.13-.331-.69-1.352-1.18-1.625-.402-.216-.977-.748-.014-.762.906-.014 1.553.834 1.769 1.179 1.035 1.74 2.688 1.25 3.349.948.1-.747.402-1.25.733-1.538-2.559-.287-5.232-1.279-5.232-5.678 0-1.25.445-2.285 1.178-3.09-.115-.288-.517-1.467.115-3.048 0 0 .963-.302 3.163 1.179.92-.259 1.897-.388 2.875-.388.977 0 1.955.13 2.875.388 2.2-1.495 3.162-1.179 3.162-1.179.633 1.581.23 2.76.115 3.048.733.805 1.179 1.825 1.179 3.09 0 4.413-2.688 5.39-5.247 5.678.417.36.776 1.05.776 2.128 0 1.538-.014 2.774-.014 3.162 0 .302.216.662.79.547C20.709 21.637 24 17.324 24 12.25 24 5.896 18.854.75 12.5.75Z"></path>
    </svg>
</a>

<a href="https://www.sciencedirect.com/science/article/abs/pii/S1568494624003788">
<img class="gh-logo" src="https://sdfestaticassets-eu-west-1.sciencedirectassets.com/shared-assets/24/images/elsevier-non-solus-new-grey.svg" alt="Elsevier logo" height="48" width="54">
<svg xmlns="http://www.w3.org/2000/svg" version="1.1" height="15" viewBox="0 0 190 23" role="img" class="gh-wordmark u-margin-s-left" aria-labelledby="gh-wm-science-direct" focusable="false" aria-hidden="true" alt="ScienceDirect Wordmark"><title id="gh-wm-science-direct">ScienceDirect</title><g><path fill="#EB6500" d="M3.81 6.9c0-1.48 0.86-3.04 3.7-3.04 1.42 0 3.1 0.43 4.65 1.32l0.13-2.64c-1.42-0.63-2.97-0.96-4.78-0.96 -4.62 0-6.6 2.44-6.6 5.45 0 5.61 8.78 6.14 8.78 9.93 0 1.48-1.15 3.04-3.86 3.04 -1.72 0-3.4-0.56-4.72-1.39l-0.36 2.64c1.55 0.76 3.57 1.06 5.15 1.06 4.26 0 6.7-2.48 6.7-5.51C12.59 11.49 3.81 10.76 3.81 6.9M20.27 9.01c0.23-0.13 0.69-0.26 1.72-0.26 1.72 0 2.41 0.3 2.41 1.58h2.38c0-0.36 0-0.79-0.03-1.09 -0.23-1.98-2.15-2.67-4.88-2.67 -3 0-6.7 2.31-6.7 7.76 0 5.22 2.77 7.99 6.63 7.99 1.68 0 3.47-0.36 4.95-1.39l-0.2-2.31c-0.99 0.82-2.84 1.52-4.06 1.52 -2.14 0-4.55-1.71-4.55-5.91C17.93 10.2 20.01 9.18 20.27 9.01"></path><rect x="29.42" y="6.97" fill="#EB6500" width="2.54" height="14.95"></rect><path fill="#EB6500" d="M30.67 0.7c-0.92 0-1.65 0.92-1.65 1.81 0 0.93 0.76 1.85 1.65 1.85 0.89 0 1.68-0.96 1.68-1.88C32.35 1.55 31.56 0.7 30.67 0.7M48.06 14.13c0-5.18-1.42-7.56-6.01-7.56 -3.86 0-6.67 2.77-6.67 7.92 0 4.92 2.97 7.82 6.73 7.82 2.81 0 4.36-0.63 5.68-1.42l-0.2-2.31c-0.89 0.79-2.94 1.55-4.69 1.55 -3.14 0-4.88-1.95-4.88-5.51v-0.49H48.06M39.91 9.18c0.17-0.17 1.29-0.46 1.98-0.46 2.48 0 3.76 0.53 3.86 3.43h-7.46C38.56 10.27 39.71 9.37 39.91 9.18zM58.82 6.57c-2.24 0-3.63 1.12-4.85 2.61l-0.4-2.21h-2.34l0.13 1.19c0.1 0.76 0.13 1.78 0.13 2.97v10.79h2.54V11.88c0.69-0.96 2.15-2.48 2.48-2.64 0.23-0.13 1.29-0.4 2.08-0.4 2.28 0 2.48 1.15 2.54 3.43 0.03 1.19 0.03 3.17 0.03 3.17 0.03 3-0.1 6.47-0.1 6.47h2.54c0 0 0.07-4.49 0.07-6.96 0-1.48 0.03-2.97-0.1-4.46C63.31 7.43 61.49 6.57 58.82 6.57M72.12 9.01c0.23-0.13 0.69-0.26 1.72-0.26 1.72 0 2.41 0.3 2.41 1.58h2.38c0-0.36 0-0.79-0.03-1.09 -0.23-1.98-2.15-2.67-4.88-2.67 -3 0-6.7 2.31-6.7 7.76 0 5.22 2.77 7.99 6.63 7.99 1.68 0 3.47-0.36 4.95-1.39l-0.2-2.31c-0.99 0.82-2.84 1.52-4.06 1.52 -2.15 0-4.55-1.71-4.55-5.91C69.77 10.2 71.85 9.18 72.12 9.01M92.74 14.13c0-5.18-1.42-7.56-6.01-7.56 -3.86 0-6.67 2.77-6.67 7.92 0 4.92 2.97 7.82 6.73 7.82 2.81 0 4.36-0.63 5.68-1.42l-0.2-2.31c-0.89 0.79-2.94 1.55-4.69 1.55 -3.14 0-4.88-1.95-4.88-5.51v-0.49H92.74M84.59 9.18c0.17-0.17 1.29-0.46 1.98-0.46 2.48 0 3.76 0.53 3.86 3.43h-7.46C83.24 10.27 84.39 9.37 84.59 9.18zM103.9 1.98h-7.13v19.93h6.83c7.26 0 9.77-5.68 9.77-10.03C113.37 7.33 110.93 1.98 103.9 1.98M103.14 19.8h-3.76V4.1h4.09c5.38 0 6.96 4.39 6.96 7.79C110.43 16.87 108.19 19.8 103.14 19.8zM118.38 0.7c-0.92 0-1.65 0.92-1.65 1.81 0 0.93 0.76 1.85 1.65 1.85 0.89 0 1.69-0.96 1.69-1.88C120.07 1.55 119.28 0.7 118.38 0.7"></path><rect x="117.13" y="6.97" fill="#EB6500" width="2.54" height="14.95"></rect><path fill="#EB6500" d="M130.2 6.6c-1.62 0-2.87 1.45-3.4 2.74l-0.43-2.37h-2.34l0.13 1.19c0.1 0.76 0.13 1.75 0.13 2.9v10.86h2.54v-9.51c0.53-1.29 1.72-3.7 3.17-3.7 0.96 0 1.06 0.99 1.06 1.22l2.08-0.6V9.18c0-0.03-0.03-0.17-0.06-0.4C132.8 7.36 131.91 6.6 130.2 6.6M145.87 14.13c0-5.18-1.42-7.56-6.01-7.56 -3.86 0-6.67 2.77-6.67 7.92 0 4.92 2.97 7.82 6.73 7.82 2.81 0 4.36-0.63 5.68-1.42l-0.2-2.31c-0.89 0.79-2.94 1.55-4.69 1.55 -3.14 0-4.89-1.95-4.89-5.51v-0.49H145.87M137.72 9.18c0.17-0.17 1.29-0.46 1.98-0.46 2.48 0 3.76 0.53 3.86 3.43h-7.46C136.37 10.27 137.52 9.37 137.72 9.18zM153.23 9.01c0.23-0.13 0.69-0.26 1.72-0.26 1.72 0 2.41 0.3 2.41 1.58h2.38c0-0.36 0-0.79-0.03-1.09 -0.23-1.98-2.14-2.67-4.88-2.67 -3 0-6.7 2.31-6.7 7.76 0 5.22 2.77 7.99 6.63 7.99 1.69 0 3.47-0.36 4.95-1.39l-0.2-2.31c-0.99 0.82-2.84 1.52-4.06 1.52 -2.15 0-4.55-1.71-4.55-5.91C150.89 10.2 152.97 9.18 153.23 9.01M170 19.44c-0.92 0.36-1.72 0.69-2.51 0.69 -1.16 0-1.58-0.66-1.58-2.34V8.95h3.93V6.97h-3.93V2.97h-2.48v3.99h-2.71v1.98h2.71v9.67c0 2.64 1.39 3.73 3.33 3.73 1.15 0 2.54-0.39 3.43-0.79L170 19.44M173.68 5.96c-1.09 0-2-0.87-2-1.97 0-1.1 0.91-1.97 2-1.97s1.98 0.88 1.98 1.98C175.66 5.09 174.77 5.96 173.68 5.96zM173.67 2.46c-0.85 0-1.54 0.67-1.54 1.52 0 0.85 0.69 1.54 1.54 1.54 0.85 0 1.54-0.69 1.54-1.54C175.21 3.13 174.52 2.46 173.67 2.46zM174.17 5.05c-0.09-0.09-0.17-0.19-0.25-0.3l-0.41-0.56h-0.16v0.87h-0.39V2.92c0.22-0.01 0.47-0.03 0.66-0.03 0.41 0 0.82 0.16 0.82 0.64 0 0.29-0.21 0.55-0.49 0.63 0.23 0.32 0.45 0.62 0.73 0.91H174.17zM173.56 3.22l-0.22 0.01v0.63h0.22c0.26 0 0.43-0.05 0.43-0.34C174 3.28 173.83 3.21 173.56 3.22z"></path></g></svg>
</a>

### Description:

- PSSM Profile: Position Specific Scoring Matrix (PSSM) is a matrix that contains the score of each amino acid at each position in the sequence. The PSSM profile is generated using PSI-BLAST.

- HMM Profile: Hidden Markov Model (HMM) is a statistical model that describes the sequence of amino acids.

- Labels: The labels are the secondary structure of the protein sequence. The secondary structure has 9 labels: 'L', 'B', 'E', 'G', 'I', 'H', 'S', 'T', 'NoSeq'


SADGRU-SS takes one-hot encoding of amino acids, PSSM, and HMM profiles as input features. The sizes of the features are `𝑛 × 22`,
`𝑛 × 21`, and `𝑛 × 20`, respectively, where 𝑛 denotes the length of the input sequence, namely `700 residues`. The one-hot encoding of amino acids and the PSSM are provided in the CB6133-filtered dataset, so these
features are not generated by hand. The HMM profile features used in this work are taken from [43], used in the MLPRNN development. The HMM profile features are generated using HHBlits [60], a software for generating HMM profiles from sequences of amino acids by searching a certain database iteratively. Lyu et al. [43] used uniclust30_2016_03.tgz database. The columns in the HMM profiles correspond to the 20 amino acid types.

## Build Model

In [4]:
def buildModel (data, do, epoch, bs) :
    sequence_length= 700
    embedding_input_dim = 22
    embedding_output_dim = 22
    pssm_input_dim = 21
    hmm_input_dim = 20
    output_dim = 9 


    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():

        ## Layers
        # input: pssm and one hot encoding
        pssm_input = Input(shape=(sequence_length, pssm_input_dim))
        embedding_input=Input(shape=(sequence_length, ))
        print("embedding input shape", embedding_input.shape)
        print("PSSM input shape", pssm_input.shape)

        hmm_input = Input(shape=(sequence_length, hmm_input_dim))
        print("HMM input shape", hmm_input.shape)

        output = Input(shape=(sequence_length, output_dim))
        print("Output shape", output.shape)

        # embedding layer for the one hot encoding
        embedding_out= Embedding(input_dim=embedding_input_dim, output_dim=embedding_output_dim, input_length=sequence_length, mask_zero=True)(embedding_input)
        print("Embedding output shape", embedding_out.shape)
        model_input =  tf.concat([embedding_out, pssm_input, hmm_input], axis=2)
        
        print("Model input shape", model_input.shape)

        ## Self attention here
        ## No Mask is used here
        
        att_output, att_scores= MultiHeadAttention(num_heads=1, key_dim=model_input.shape[-1], dropout=0.5)(query=model_input, value=model_input, return_attention_scores=True)
        model_input_att = Add()([model_input, att_output])
        model_input_att = LayerNormalization()(model_input_att)
        model_input_att = BatchNormalization(synchronized=True)(model_input_att)

        ## Block 1

        model_input1 = Dense(63, activation='relu')(model_input_att)
        model_input2 = Dense(128, activation='relu')(model_input1)

        whole_sequence_output1, final_state_1f, final_state_1b = Bidirectional(GRU(128, return_state=True, return_sequences=True, dropout=0.3))(model_input2)
        whole_sequence_output2, final_state_2f, final_state_2b = Bidirectional(GRU(128, return_state=True, return_sequences=True, dropout=0.3))(whole_sequence_output1)

        whole_sequence_output2=BatchNormalization(synchronized=True)(whole_sequence_output2)

        ## Block 2

        model_input3 = Dense(63, activation='relu')(model_input_att)
        model_input4 = Dense(256, activation='relu')(model_input3)

        whole_sequence_output3, final_state_3f, final_state_3b = Bidirectional(GRU(256, return_state=True, return_sequences=True, dropout=0.5))(model_input4)
        whole_sequence_output4, final_state_4f, final_state_4b = Bidirectional(GRU(256, return_state=True, return_sequences=True, dropout=0.5))(whole_sequence_output3)

        whole_sequence_output4=BatchNormalization(synchronized=True)(whole_sequence_output4)

        ## Block 3
        model_input5 = Dense(63, activation='relu')(model_input_att)
        model_input6 = Dense(512, activation='relu')(model_input5)

        whole_sequence_output5, final_state_5f, final_state_5b = Bidirectional(GRU(512, return_state=True, return_sequences=True, dropout=0.6))(model_input6)
        whole_sequence_output6, final_state_6f, final_state_6b = Bidirectional(GRU(512, return_state=True, return_sequences=True, dropout=0.6))(whole_sequence_output5)

        whole_sequence_output6=BatchNormalization(synchronized=True)(whole_sequence_output6)

        ## Combine the blocks

        whole_sequence_output = tf.concat([whole_sequence_output2, whole_sequence_output4, whole_sequence_output6], axis=-1)
        whole_sequence_output = Dropout(do)(whole_sequence_output)
        
      
        # Fully Connected Layers

        dense0= Dense(512, activation='relu')(whole_sequence_output)
        dense1= Dense(256, activation='relu')(dense0)
        dense2= Dense(128, activation='relu')(dense1)
        
        out = TimeDistributed(Dense(output.shape[2], activation='softmax'))(dense2)


        print("Out shape", out.shape)

        ## MODEL TRAINING
        model = Model(inputs=[embedding_input, pssm_input, hmm_input], outputs=out)
        opt = Adam(learning_rate=0.0001)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()

    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

    epc = epoch
    es = EarlyStopping(monitor='val_loss', mode='min', patience=10)
    history = model.fit([data["sequence"], data["profile"], data["hmm_profile"]], data["label"],
                    epochs=epc, verbose=1, callbacks=[es], validation_split=0.1,
                    batch_size=bs)

    #Plot the training loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')


    # Save the model
    model_name = 'model'
    model.save(model_name+'.h5')



In [ ]:
# Call prepareData function
train_data= prepareData()

# Call buildModel function
buildModel(train_data, 0.5, 100, 64)


Shape of amino acid sequence (before transform): (5534, 700, 22)
Shape of amino acid sequence (after transform): (5534, 700)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
(None, 700)
(None, 700, 21)
(None, 700, 20)
(None, 700, 9)
(None, 700, 22)
(None, 700, 63)
(None, 700, 9)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 700)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 700, 22)      484         ['input_2[0][0]']                
                                                                                                  
 input_1 (InputLayer)           [(None, 700, 21)]    0    